# Financial Impact Analysis: Proving the Value of Optimization

This notebook conducts a comparative financial analysis for the Flower Warehouse project. 

**Objective**: To quantitatively prove that the implemented **First-In, First-Out (FIFO)** fulfillment strategy is financially superior to a sub-optimal **Last-In, First-Out (LIFO)** strategy by measuring its impact on spoilage costs and net profit.

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import warnings
import io

warnings.filterwarnings('ignore')

## 1. Load Simulated Data

Instead of connecting to a live database, we will simulate the data as if it were extracted from the running pipeline. This makes the analysis self-contained and reproducible.

In [ ]:
# --- Mock Data Simulation ---
# This data represents the state of the warehouse after the pipeline has run for some time.

# Flower Catalog Data
catalog_data = """
flower_type,required_temp_celsius,shelf_life_days,cost_per_unit,price_per_unit
Rose,4,7,1.50,3.00
Tulip,4,5,0.75,2.00
Lily,8,6,2.00,4.50
Orchid,20,10,5.00,12.00
Daisy,8,5,0.50,1.50
Sunflower,8,6,1.00,2.50
"""

# Current Inventory State (Note the different arrival dates)
inventory_data = """
shipment_id,flower_type,quantity,arrival_date,bin_id
SH101,Rose,100,2025-08-05,A01
SH102,Rose,50,2025-08-14,A02
SH103,Tulip,120,2025-08-08,B01
SH104,Lily,80,2025-08-13,C01
"""

# Fulfilled Picking Tasks (Corrected to include flower type in details)
tasks_data = """
task_id,task_type,details,timestamp
1,PICKING,"For Order ORD911, Pick 50 Rose from Bin A01",2025-08-15 10:00:00
2,PICKING,"For Order ORD913, Pick 10 Tulip from Bin B01",2025-08-15 10:00:00
"""

catalog_df = pd.read_csv(io.StringIO(catalog_data))
inventory_df = pd.read_csv(io.StringIO(inventory_data))
tasks_df = pd.read_csv(io.StringIO(tasks_data))

print("✅ Mock data successfully loaded into DataFrames.")

# Add cost and price to the main inventory dataframe for calculations
inventory_df = inventory_df.merge(catalog_df, on='flower_type')

## 2. Analyze the Optimized (FIFO) Strategy

We will calculate the total revenue from fulfilled orders and the total cost of any inventory that expired and was therefore spoiled based on the FIFO strategy.

In [ ]:
# --- CORRECTED LOGIC --- 
# First, create a copy of the inventory to simulate fulfillment on.
fifo_inventory = inventory_df.copy()
fifo_revenue = 0

# Reconstruct the orders that were fulfilled from the picking tasks
orders = {}
for _, task in tasks_df.iterrows():
    parts = task['details'].split()
    order_id = parts[2].replace(',', '')
    quantity = int(parts[4])
    flower_type = ' '.join(parts[5:-3])
    if order_id not in orders:
        orders[order_id] = {'flower_type': flower_type, 'quantity': 0}
    orders[order_id]['quantity'] += quantity

# Simulate the FIFO fulfillment process
for order in orders.values():
    stock_for_order = fifo_inventory[fifo_inventory['flower_type'] == order['flower_type']].copy()
    if not stock_for_order.empty and stock_for_order['quantity'].sum() >= order['quantity']:
        stock_for_order['arrival_date'] = pd.to_datetime(stock_for_order['arrival_date'])
        stock_for_order = stock_for_order.sort_values('arrival_date', ascending=True) # Sort by oldest first
        
        qty_to_fulfill = order['quantity']
        for index, stock_batch in stock_for_order.iterrows():
            if qty_to_fulfill > 0:
                pick_qty = min(qty_to_fulfill, stock_batch['quantity'])
                fifo_inventory.loc[fifo_inventory.index == index, 'quantity'] -= pick_qty
                qty_to_fulfill -= pick_qty
        
        if qty_to_fulfill == 0:
            price = catalog_df[catalog_df['flower_type'] == order['flower_type']]['price_per_unit'].iloc[0]
            fifo_revenue += order['quantity'] * price

# Now, calculate spoilage cost from the REMAINING inventory
fifo_inventory['arrival_date'] = pd.to_datetime(fifo_inventory['arrival_date'])
fifo_inventory['expiration_date'] = fifo_inventory['arrival_date'] + pd.to_timedelta(fifo_inventory['shelf_life_days'], unit='d')
spoiled_fifo_inventory = fifo_inventory[fifo_inventory['expiration_date'] < datetime.now()]
fifo_spoilage_cost = (spoiled_fifo_inventory['quantity'] * spoiled_fifo_inventory['cost_per_unit']).sum()

fifo_profit = fifo_revenue - fifo_spoilage_cost

print("✅ FIFO (Optimized) scenario calculated.")

## 3. Simulate the Sub-Optimal (LIFO) Strategy

Now, we simulate what would have happened if the warehouse had used a LIFO strategy (selling the newest stock first) to fulfill the exact same set of orders. We will use a copy of the inventory and calculate the resulting spoilage.

In [ ]:
lifo_inventory = inventory_df.copy()
lifo_revenue = 0

for order in orders.values():
    stock_for_order = lifo_inventory[lifo_inventory['flower_type'] == order['flower_type']].copy()
    if not stock_for_order.empty and stock_for_order['quantity'].sum() >= order['quantity']:
        # Sort by newest first (LIFO)
        stock_for_order['arrival_date'] = pd.to_datetime(stock_for_order['arrival_date'])
        stock_for_order = stock_for_order.sort_values('arrival_date', ascending=False)
        
        qty_to_fulfill = order['quantity']
        for index, stock_batch in stock_for_order.iterrows():
            if qty_to_fulfill > 0:
                pick_qty = min(qty_to_fulfill, stock_batch['quantity'])
                lifo_inventory.loc[lifo_inventory.index == index, 'quantity'] -= pick_qty
                qty_to_fulfill -= pick_qty
        
        if qty_to_fulfill == 0:
            price = catalog_df[catalog_df['flower_type'] == order['flower_type']]['price_per_unit'].iloc[0]
            lifo_revenue += order['quantity'] * price

# Calculate spoilage for the LIFO simulation
lifo_inventory['arrival_date'] = pd.to_datetime(lifo_inventory['arrival_date'])
lifo_inventory['expiration_date'] = lifo_inventory['arrival_date'] + pd.to_timedelta(lifo_inventory['shelf_life_days'], unit='d')
spoiled_lifo_inventory = lifo_inventory[lifo_inventory['expiration_date'] < datetime.now()]
lifo_spoilage_cost = (spoiled_lifo_inventory['quantity'] * spoiled_lifo_inventory['cost_per_unit']).sum()
lifo_profit = lifo_revenue - lifo_spoilage_cost

print("✅ LIFO (Sub-Optimal) scenario simulated.")

## 4. Comparative Financial Results

In [ ]:
results = {
    'Metric': ['Total Revenue', 'Spoilage Cost', 'Net Profit'],
    'Optimized (FIFO)': [f"${fifo_revenue:,.2f}", f"${fifo_spoilage_cost:,.2f}", f"${fifo_profit:,.2f}"],
    'Sub-Optimal (LIFO)': [f"${lifo_revenue:,.2f}", f"${lifo_spoilage_cost:,.2f}", f"${lifo_profit:,.2f}"]
}
results_df = pd.DataFrame(results).set_index('Metric')

profit_improvement = fifo_profit - lifo_profit
cost_reduction = lifo_spoilage_cost - fifo_spoilage_cost

print("="*50)
print("  Financial Impact Analysis: FIFO vs. LIFO")
print("="*50)
display(results_df)
print("="*50)

print("\n--- Conclusion ---")
print(f"📈 The optimized FIFO strategy increased profit by: ${profit_improvement:,.2f}")
print(f"💸 The optimized FIFO strategy reduced spoilage costs by: ${cost_reduction:,.2f}")